[Reference](https://medium.com/@atarax/finally-json-table-is-here-postgres-17-a9b5245649bd)

# Set up Postgres locally using Docker

## Step 1: Create Initialization Scripts

```
mkdir -p postgres-init
```

### Inside postgres-init, create the following two files:
#### 1. 01-create_role.sh: Script to create a role in PostgreSQL.
```
#!/bin/bash
psql -v ON_ERROR_STOP=1 --username "$POSTGRES_USER" --dbname "$POSTGRES_DB" <<-EOSQL
  CREATE ROLE myrole WITH LOGIN PASSWORD 'mypassword';
EOSQL
```

## 2. 02-create_table_schema_and_insert.sh: Script to create a schema or any other necessary database setup.
```
#!/bin/bash
set -e

echo "creating schema"

psql -v ON_ERROR_STOP=1 --username "$POSTGRES_USER" --dbname "postgres" <<-EOSQL

 CREATE TABLE IF NOT EXISTS public.products (
 id integer NOT NULL,
 product_name varchar NOT NULL,
 data jsonb
 );

 INSERT INTO public.products (id, product_name, data)
 VALUES
  (1, 'iPhone 16', jsonb_build_object(
  'brand', 'Apple',
  'model', '16',
  'release_year', 2024,
  'specifications', jsonb_build_object(
   'screen_size', '6.7 inches',
   'processor', 'Apple A18 Bionic',
   'ram', '8 GB',
   'storage', '256 GB',
   'battery', '4500 mAh',
   'camera', jsonb_build_object(
    'rear', jsonb_build_array(
     jsonb_build_object('type', 'Wide', 'megapixels', 108),
     jsonb_build_object('type', 'Ultra-wide', 'megapixels', 12),
     jsonb_build_object('type', 'Telephoto', 'megapixels', 12)
    ),
    'front', jsonb_build_object('type', 'Wide', 'megapixels', 32)
   )
  ),
  'features', jsonb_build_array(
   '5G capable',
   'Water-resistant (IP68)',
   'Wireless charging',
   'Fast charging support',
   'Face ID',
   'ProMotion 120Hz display'
  ),
  'warranty', '2 years',
  'price', 1199.99
  )),
  (2, 'Macbook Pro 2023', jsonb_build_object(
   'brand', 'Apple',
   'model', 'Pro 2023',
   'release_year', 2023,
   'specifications', jsonb_build_object(
    'screen_size', '15.6 inches',
    'processor', 'M3 pro',
    'ram', '16 GB',
    'storage', '512 GB SSD',
    'graphics_card', 'M3 pro',
    'battery', 'Up to 14 hours'
   ),
   'features', jsonb_build_array(
    'Backlit keyboard',
    'Fingerprint reader',
    'Thunderbolt 4 ports',
    'Retina display'
   ),
   'warranty', '1 year',
   'price', 1499.99
  )),
  (3, 'Sony Headphones Pro', jsonb_build_object(
   'brand', 'AudioSound',
   'model', 'Sony WH-CH720n',
   'release_year', 2021,
   'specifications', jsonb_build_object(
    'connectivity', 'Bluetooth 5.0',
    'battery_life', '35 hours',
    'driver_size', '40mm',
    'weight', '250 grams'
   ),
   'features', jsonb_build_array(
    'Active noise cancellation',
    'Button controls',
    'Voice assistant integration',
    'Water-resistant design'
   ),
   'warranty', '1 year',
   'price', 299.99
  ));


EOSQL
```


### Step 2: Create docker-compose.yml
#### Create docker-compose.yml file in your root directory
```
services:
  db1:
    image: postgres:17.0
    environment:
      POSTGRES_DB: postgres
      POSTGRES_USER: admin
      POSTGRES_PASSWORD: dbpassword
    ports:
      - "5432:5432"
    command: postgres -c 'max_connections=1000'
    volumes:
      - ./postgres-init/01-create_role.sh:/docker-entrypoint-initdb.d/01-create_role.sh
      - ./postgres-init/02-create_table_schema_and_insert.sh:/docker-entrypoint-initdb.d/02-create_table_schema_and_insert.sh
```

### Step 3: Start the PostgreSQL Service
```
docker-compose up -d
```

### Step 4: Verify if the container is up and running
```
docker ps
```

 # Connect with your DB
Set up the values as above to connect to your Postgres Db.
- Host: localhost
- Database: postgres
- Username: admin
- Password: dbpassword

Find the name or ID of the container running the PostgreSQL database. You can use the docker ps command to list all running containers and their IDs and names.
```
docker ps
```

Use the docker exec command to start a shell inside the container. Replace “container_name_or_id” with the name or ID of the container.

```
docker exec -it container_or_id bash
```

Once you are inside the container’s shell, run the psql command to connect to the PostgreSQL database. Replace “username” and “database_name” with your database credentials.

```
psql -U username -d database_name
```



# Run SQL queries
```
SELECT jt.*
FROM public.products,
     JSON_TABLE(
         data,
         '$'
         COLUMNS (
             brand text PATH '$.brand',
             model text PATH '$.model',
             release_year integer PATH '$.release_year',
             specifications jsonb PATH '$.specifications',
             features jsonb PATH '$.features',
             warranty text PATH '$.warranty',
             price numeric PATH '$.price'
         )
     ) AS jt;
```